In [1]:
import ROOT
from ROOT import *

Welcome to JupyROOT 6.24/02


In [2]:
##For Visualization
%jsroot on

 Get the data and background histogram files
 
 Make a dictionary contains of data and MC histo files for later

In [3]:
inputfilepath='../inputfiles/'

WorkdayTag= '_Sep6_v0'

AnalysisName= 'VLL2018_'

dataAndMCbundle=['Data','wjets','zjets','ttjets','tW','STtchannelTop','WW2Q','QCDMultijet']
FileName=['data', 'wjets', 'zjets', 'ttjets', 'tW', 'ST', 'WW2Q','QCD']
FilesDict={}
for ifile,file in enumerate(dataAndMCbundle):
    file_ifile=TFile.Open(inputfilepath+AnalysisName+dataAndMCbundle[ifile]+WorkdayTag+'.root','READ')
    FilesDict[FileName[ifile]]=file_ifile
        
FilesDict

{'data': <cppyy.gbl.TFile object at 0x5142600>,
 'wjets': <cppyy.gbl.TFile object at 0x37cd2c0>,
 'zjets': <cppyy.gbl.TFile object at 0x454d060>,
 'ttjets': <cppyy.gbl.TFile object at 0x45d9080>,
 'tW': <cppyy.gbl.TFile object at 0x45d9800>,
 'ST': <cppyy.gbl.TFile object at 0x226f6c0>,
 'WW2Q': <cppyy.gbl.TFile object at 0x2231f70>,
 'QCD': <cppyy.gbl.TFile object at 0x8b8afd0>}

In [4]:
CrossSectionDB={
    'data':1,
    'wjets':50380,
    'zjets':5765,
    'ttjets':365.34,
    'tW':34.91,
    'ST':136.02,
    'WW2Q':45.68,
    'QCD':239400    
}

OverlayColorScheme={
    'data':kBlack,
    'wjets':kGray+1,
    'zjets':kRed-9,
    'ttjets':kCyan-10,
    'tW':kBlue-9,
    'ST':kBlue-7,
    'WW2Q':kGreen-4,
    'QCD':kYellow    
}

StackingColorScheme={
    'data':kBlack,
    'wjets':kGray+1,
    'zjets':kRed-9,
    'ttjets':kCyan-10,
    'tW':kBlue-9,
    'ST':kBlue-5,
    'WW2Q':kGreen-10,
    'QCD':kYellow-10    
}                 

Lets take one variable for plotting
 - **Type:**  " ","_highMVA","_regA","_regB","_regC","_regD"
 

In [5]:
var='mtjet0'
Type=["","_highMVA","_regA","_regB","_regC","_regD"]
Description=['Basic WjetsCR','High MVA region','Region A','Region B','Region C','Region D']
Plotname = var+Type[0]+'_l2j'
print('We want to plot '+ Plotname + ' at '+ Description[0])

We want to plot mtjet0_l2j at Basic WjetsCR


In [6]:
FilesDict['data'].Get(Plotname)

In [7]:
#Count Events in histogram
def Count(histo):                                                               
    n_count="{:.3f}".format(histo.Integral())                                   
    return str(n_count) 

Let get the corresponding histogram from their TFiles

In [8]:
HistoDict={}
histogram=[]
dataLumi=59.7*1000 #in pb

for index,(filekey,file) in enumerate(FilesDict.items()):
    #print(filekey,file)
    histo= file.Get(Plotname)
    histo.SetDirectory(0)
    histoevent=float(Count(histo))
    
    #LumiCorr-------------------------------
    totEvent=file.nEvents.GetBinContent(2)
    Xsec=CrossSectionDB[filekey]
    sampleLumi=totEvent/Xsec
    if(filekey!='data'): lumi_corr=dataLumi/sampleLumi
    else: lumi_corr=1
    #print(filekey,totEvent,Xsec,sampleLumi,lumi_corr)    
    histo.Scale(lumi_corr)
    histoeventAfterCorr=float(Count(histo))
    #---------------------------------------
    histogram.append([filekey,histoevent,histoeventAfterCorr,OverlayColorScheme[filekey],histo])
    HistoDict[filekey]=[filekey,histoevent,histoeventAfterCorr,OverlayColorScheme[filekey],histo]
    

In [9]:
histogram

[['data', 2200913.0, 2200913.0, 1, <cppyy.gbl.TH1F object at 0x8bcc510>],
 ['wjets', 24415.0, 1279260.417, 921, <cppyy.gbl.TH1F object at 0x8d36d40>],
 ['zjets', 32652.0, 112696.473, 623, <cppyy.gbl.TH1F object at 0x6b0db10>],
 ['ttjets', 417770.0, 99024.552, 422, <cppyy.gbl.TH1F object at 0x3764700>],
 ['tW', 45837.0, 9953.129, 591, <cppyy.gbl.TH1F object at 0x3775780>],
 ['ST', 7040.0, 4095.597, 593, <cppyy.gbl.TH1F object at 0x6b0a3e0>],
 ['WW2Q', 41762.0, 24318.957, 412, <cppyy.gbl.TH1F object at 0x37748b0>],
 ['QCD', 1018.0, 656405.56, 400, <cppyy.gbl.TH1F object at 0x8d70740>]]

In [10]:
def SortDict(dic,opt):
    return dict(sorted(dic.items(),key=lambda x:x[1][1],reverse=opt))    

In [11]:
HistoDict=SortDict(HistoDict,True) ## Sort the dict in terms of no of events
                                   ## Sort(dic,True)==Descending order 
                                   ## Sort(dic,False)==Ascending order

In [12]:
HistoDict.keys()

dict_keys(['data', 'ttjets', 'tW', 'WW2Q', 'zjets', 'wjets', 'ST', 'QCD'])

Make a copy of this dictionary to plot overlay histogram

In [13]:
import copy
HistoDictForOverlay=copy.deepcopy(HistoDict)

## Lets overlay them for shape study

In [14]:
import plot_settings as plotsetting

plotsetting.PlotRatioPad=False
plotsetting.year=2018                                                                                                           
plotsetting.publicationStyle=True
plotsetting.WorkInProgress=True

In [15]:
#bkgBundle=['wjets', 'zjets', 'ttjets', 'tW', 'ST', 'WW2Q','QCD']
bkgBundle=list(HistoDictForOverlay.keys())[1:]

In [25]:
canvas=TCanvas("canvas","",700,600)
mainpad=plotsetting.CreatePad("mainpad",0,1)
mainpad.SetLogy(0)
mainpad.Draw()
mainpad.cd()
#Legend
legend=TLegend(0.60,0.55,0.90,0.82)
plotsetting.SetLegendStyle(legend)
for imc,mc in enumerate(bkgBundle):
    histoname=HistoDictForOverlay[mc][0]
    color=HistoDictForOverlay[mc][3]
    histo=HistoDictForOverlay[mc][-1]
    histo.Draw("HIST SAME")
    histo.Rebin(50)
    histo.Scale(1/float(Count(histo)))
    plotsetting.SetOverlayHistoStyle(histo,color,2,"HT (GeV)")
    #histo.SetLineColor(color)
    #histo.SetLineWidth(2)
    maxbin=histo.GetMaximumBin()
    maxheight=histo.GetBinContent(maxbin)
    histo.GetYaxis().SetRangeUser(0.001,(maxheight*2.6))
    #histo.GetYaxis().SetTitle('Normalized Count')
    
    print(histo.GetBinContent(maxbin))
    #histo.SetMarkerStyle(20)
    legend.AddEntry(histo,histoname,'l')
legend.Draw()

    
canvas.Draw()

0.29673296213150024
0.24936187267303467
0.2492217868566513
0.23761878907680511
0.4444444477558136
0.5294695496559143
0.3136596381664276


Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas
Error in <TH1F::Rebin>: Illegal value of ngroup=50
Error in <TH1F::Rebin>: Illegal value of ngroup=50
Error in <TH1F::Rebin>: Illegal value of ngroup=50
Error in <TH1F::Rebin>: Illegal value of ngroup=50
Error in <TH1F::Rebin>: Illegal value of ngroup=50
Error in <TH1F::Rebin>: Illegal value of ngroup=50
Error in <TH1F::Rebin>: Illegal value of ngroup=50


In [17]:
import plot_settings as plotsetting
plotsetting.PlotRatioPad=True
plotsetting.year=2018                                                                                                           
plotsetting.publicationStyle=False
plotsetting.WorkInProgress=True

## Let's do Stack Histogramming and Plot data

Just check the Dictionary to see all component is present

In [18]:
HistoDict.values()

dict_values([['data', 2200913.0, 2200913.0, 1, <cppyy.gbl.TH1F object at 0x8bcc510>], ['ttjets', 417770.0, 99024.552, 422, <cppyy.gbl.TH1F object at 0x3764700>], ['tW', 45837.0, 9953.129, 591, <cppyy.gbl.TH1F object at 0x3775780>], ['WW2Q', 41762.0, 24318.957, 412, <cppyy.gbl.TH1F object at 0x37748b0>], ['zjets', 32652.0, 112696.473, 623, <cppyy.gbl.TH1F object at 0x6b0db10>], ['wjets', 24415.0, 1279260.417, 921, <cppyy.gbl.TH1F object at 0x8d36d40>], ['ST', 7040.0, 4095.597, 593, <cppyy.gbl.TH1F object at 0x6b0a3e0>], ['QCD', 1018.0, 656405.56, 400, <cppyy.gbl.TH1F object at 0x8d70740>]])

In [19]:
##Test
#print(Count(HistoDict['wjets'][-1]))
if(float(Count(HistoDict['wjets'][-1]))<10):
    print("**************")
    print("Caution!!!!!!!!!!!!!!!! The histograms in dictionary is already normalized to 1")
    print("Please make a deep copy of the master dictionary and use them")
    print("**************")
else:
    print("Wjets= ",Count(HistoDict['wjets'][-1]))

Wjets=  1279260.417


In [20]:
###Sorting based on lumi corrected weight
HistoDict=dict(sorted(HistoDict.items(),key=lambda x:x[1][2],reverse=False))
print("         Summary Report                ")
print("_________________________________________")
print("   ")
print( "  Name"+"     "+"    No of Events ")
print("--------"+"    "+"-------------------")
exp=0
for key in HistoDict.keys():
    if(key!='data'): exp=exp+HistoDict[key][2]
    print(" "+ key+"          " +str(HistoDict[key][2]))
print("_________________________________________")
print("    ")
print("obs="+str(HistoDict['data'][2])+", "+"exp="+str(exp))
print("obs/exp=",str((HistoDict['data'][2]/exp))[:5])
print("_________________________________________")
print("                                         ")
obs=HistoDict['data'][2]
bkgBundle=list(HistoDict.keys())[:-1]
print("Background Bundle=",bkgBundle)    

         Summary Report                
_________________________________________
   
  Name         No of Events 
--------    -------------------
 ST          4095.597
 tW          9953.129
 WW2Q          24318.957
 ttjets          99024.552
 zjets          112696.473
 QCD          656405.56
 wjets          1279260.417
 data          2200913.0
_________________________________________
    
obs=2200913.0, exp=2185754.685
obs/exp= 1.006
_________________________________________
                                         
Background Bundle= ['ST', 'tW', 'WW2Q', 'ttjets', 'zjets', 'QCD', 'wjets']


Now Lets make the plot

In [21]:
def PlotMaker(Dictionary,plotname,Xaxislabel,rebin): ## Give the dictionary of histograms
    Dict=copy.deepcopy(Dictionary)
    Dict=dict(sorted(Dict.items(),key=lambda x:x[1][2],reverse=False))
    #Background Bundle
    bkgBundle=list(Dict.keys())[:-1]
    print(bkgBundle)
    
    rebinning=rebin
    #DataHisto
    dataHisto=Dict['data'][-1]
    dataHisto.Rebin(rebinning)
    plotsetting.SetOverflowBin(dataHisto)
        
    ##Legend Settings
    legend = ROOT.TLegend(0.99, 0.45, 0.80,0.86) 
    plotsetting.SetLegendStyle(legend)
    legend.AddEntry(dataHisto,"Data ["+plotsetting.Count(dataHisto)+"]","ep")
    for imc,mc in reversed(list(enumerate(bkgBundle))):
        histoname=Dict[mc][0]
        color=Dict[mc][3]
        histo=Dict[mc][-1]    
        legend.AddEntry(histo,histoname+"["+Count(histo)+"]","lf")
        
    ##
    ##Stacking(ALL MC Backgrounds)
    stack=THStack()
    ratioHisto=dataHisto.Clone()
    h_Allbkg=Dict['ST'][-1].Clone()
    h_Allbkg.Rebin(rebinning)
    for imc,mc in enumerate(bkgBundle):
        histoname=Dict[mc][0]
        color=Dict[mc][3]
        histo=Dict[mc][-1]    
        ##SetStyle
        histo.SetLineWidth(0)                                                                                                       
        histo.SetLineColor(1)                                                                                                       
        histo.SetFillColor(color)                                                                                             
        histo.Rebin(rebinning)
        plotsetting.SetOverflowBin(histo)
        stack.Add(histo)                                                                                                            
        #Denominotor for ratioPlot
        if(imc>0): h_Allbkg.Add(histo)
    ratioHisto.Divide(h_Allbkg)
    
    
    obs=float(Count(dataHisto))
    exp=float(Count(h_Allbkg))
    ##Legend Settings
    legend = ROOT.TLegend(0.99, 0.45, 0.80,0.86) 
    plotsetting.SetLegendStyle(legend)
    legend.AddEntry(dataHisto,"Data ["+plotsetting.Count(dataHisto)+"]","ep")
    for imc,mc in reversed(list(enumerate(bkgBundle))):
        histoname=Dict[mc][0]
        color=Dict[mc][3]
        histo=Dict[mc][-1]    
        legend.AddEntry(histo,histoname+"["+Count(histo)+"]","lf")
    ##Obs/Exp Legend
    ratioleg = ROOT.TLegend(0.99, 0.90, 0.81,0.87)
    plotsetting.SetLegendStyle(ratioleg)
    ratioleg.SetTextSize(0.03)
    ratioleg.SetHeader("obs/exp="+(str(obs/exp)[:5])) 
    
    ##Plotting
    canvas=TCanvas("c","",700,600)                                                                                             
    pad=plotsetting.CreatePad("pad",0,1.0)                                                                                          
    #pad.SetLogy(0)
    pad.Draw()                                                                                                                      

    #RatioPad                                                                                                                       
    if(plotsetting.PlotRatioPad==True):                                                                                                         
        ratioPad = plotsetting.CreateRatioPad('ratioPad',0,0.25)                                                                    
        ratioPad.SetGrid(1)                                                                                                         
        ratioPad.Draw()                                                                                                             

    pad.cd()                                                                                                                        
    ##MainPadStyle 
    pad.SetFillStyle(4000)
    plotsetting.SetHistoStyle(dataHisto,xlabel=plotname)
    dataHisto.GetYaxis().SetRangeUser(0.1,1e7)
    ##Draw                                                                                                                          
    dataHisto.Draw("ep")                                                                                                            
    stack.Draw("HIST SAME")                                                                                                         
    dataHisto.Draw("ep same")                                                                                                       
    legend.Draw("SAME")
    ratioleg.Draw("SAME")
    plotsetting.CMSLabel(pad)
    pad.SetTickx(1)
    pad.Update()
    ##RatioPadDraw                                                                                                                  
    if(plotsetting.PlotRatioPad==True):                                                                                                         
        ratioPad.cd()                                                                                                               
        plotsetting.SetRatioHistoStyle(ratioHisto,xlabel=Xaxislabel)                                                              
        ratioHisto.Draw('ep')

    #Extra Text
    pad.cd()
    plotsetting.DrawText(0.60,0.84,'2018 WJets CR')    
    
    canvas.Draw()
    
    return canvas,stack,legend,ratioHisto,ratioleg

In [22]:
PlotMaker(HistoDict,'mtjet01','Leading Jet M_{T}',100)

['ST', 'tW', 'WW2Q', 'ttjets', 'zjets', 'QCD', 'wjets']


(<cppyy.gbl.TCanvas object at 0x9349040>,
 <cppyy.gbl.TLegend object at 0x969c660>)